In [1]:
# Ch02-2 - Pitfalls of joining data with pandas

In [2]:
# Libraries
import pandas as pd

In [4]:
# Jumble the data using random sampling
vdata = pd.read_csv("data/2021VAERSDATA.csv.gz", encoding="iso-8859-1") 
vdata.sample(frac=0.9).to_csv("vdata_sample.csv.gz", index=False) 
vax = pd.read_csv("data/2021VAERSVAX.csv.gz", encoding="iso-8859-1") 
vax.sample(frac=0.9).to_csv("vax_sample.csv.gz", index=False) 
# Note - it is ok to get a dtype warning here

/var/folders/53/kmyyy3057lndfb0bpwx_2pkr0000gn/T/ipykernel_43783/2610587055.py:2: DtypeWarning: Columns (7,12,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  vdata = pd.read_csv("data/2021VAERSDATA.csv.gz", encoding="iso-8859-1")


In [5]:
# Inner join on the tables
vdata = pd.read_csv("vdata_sample.csv.gz", low_memory=False) 
vax = pd.read_csv("vax_sample.csv.gz", low_memory=False) 
vdata_with_vax = vdata.join(vax.set_index("VAERS_ID"), on="VAERS_ID", how="inner") 
len(vdata), len(vax), len(vdata_with_vax) 

(677736, 717399, 645567)

In [8]:
# Find the data not captured by the join
lost_vdata = vdata.loc[~vdata.index.isin(vdata_with_vax.index)] 
lost_vdata 
lost_vax = vax[~vax["VAERS_ID"].isin(vdata.index)] 
lost_vax 

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,1186337,COVID19,MODERNA,026b21a,1,IM,LA,COVID19 (COVID19 (MODERNA))
1,943580,COVID19,MODERNA,039120A,1,IM,RA,COVID19 (COVID19 (MODERNA))
2,1176243,COVID19,MODERNA,019B21A,2,IM,LA,COVID19 (COVID19 (MODERNA))
3,1255767,DTAPIPVHIB,SANOFI PASTEUR,UJ162AB,3,OT,NaN,DTAP + IPV + HIB (PENTACEL)
4,1269613,COVID19,MODERNA,44A21A,1,IM,LA,COVID19 (COVID19 (MODERNA))
...,...,...,...,...,...,...,...,...
717394,991070,COVID19,PFIZER\BIONTECH,EL9262,1,SYR,RA,COVID19 (COVID19 (PFIZER-BIONTECH))
717395,1727097,COVID19,MODERNA,038C21A,UNK,IM,LA,COVID19 (COVID19 (MODERNA))
717396,1222179,COVID19,PFIZER\BIONTECH,en6200,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
717397,952505,COVID19,PFIZER\BIONTECH,EL3247,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))


In [6]:
# Left outer join
vdata_with_vax_left = vdata.join(vax.set_index("VAERS_ID"), on="VAERS_ID") 
vdata_with_vax_left.groupby("VAERS_ID").size().sort_values() 

VAERS_ID
910642      1
1595074     1
1595075     1
1595076     1
1595079     1
           ..
1931705     9
1714163     9
1147077    10
1147080    11
962303     12
Length: 677736, dtype: int64

In [8]:
# Right join
dead = vdata[vdata.DIED == "Y"] 
vax19 = vax[vax.VAX_TYPE == "COVID19"] 
vax19_dead = vax19.join(dead.set_index("VAERS_ID"), on="VAERS_ID", how="right") 
len(vax19), len(dead), len(vax19_dead) 
len(vax19_dead[vax19_dead.VAERS_ID.duplicated()]) 
len(vax19_dead) - len(dead) 

1251

In [9]:
# Problematic lots
vax19_dead["STATE"] = vax19_dead["STATE"].str.upper() 
dead_lot = vax19_dead[["VAERS_ID", "VAX_LOT", "STATE"]].set_index(["VAERS_ID", "VAX_LOT"]) 
dead_lot_clean = dead_lot[~dead_lot.index.duplicated()] 
dead_lot_clean = dead_lot_clean.reset_index() 
dead_lot_clean[dead_lot_clean.VAERS_ID.isna()] 
baddies = dead_lot_clean.groupby("VAX_LOT").size().sort_values(ascending=False) 

In [10]:
# Print problematic lots
for i, (lot, cnt) in enumerate(baddies.items()):
    print(lot, cnt, len(dead_lot_clean[dead_lot_clean.VAX_LOT == lot].groupby("STATE")))
    if i == 10:
        break

Unknown 176 32
EN6201 129 29
unknown 99 20
EN6200 99 19
EN5318 95 23
EN6198 85 18
EL9261 85 18
EN6202 80 19
EL9269 80 19
EL3248 79 14
EM9810 78 19


In [ ]:
# End of Notebook #